In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

In [2]:
ori_data = np.load('../Data/samples_24/stock_norm_truth.npy')
fake_data = np.load('../Data/ldm_fake_stock_12_2_16_3.npy')
iterations = 5

In [3]:
## Context-FID Score
context_fid_score = []
for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.03986386624434843 

Iter 1:  context-fid = 0.028533125116510037 

Iter 2:  context-fid = 0.030732085798004342 

Iter 3:  context-fid = 0.026176991295331395 

Iter 4:  context-fid = 0.0333870527561559 

Final Score:  0.031738624242070024 ± 0.006539603328318726


In [4]:
## Correlational Score
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.020072271484621408 

Iter 1:  cross-correlation = 0.03865088127922828 

Iter 2:  cross-correlation = 0.024644700707747248 

Iter 3:  cross-correlation = 0.027294691526499205 

Iter 4:  cross-correlation = 0.028937693636441407 

Final Score:  0.02792004772690751 ± 0.008532611470639814


In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

In [6]:
### Discriminative score
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use tf.global_variables instead.


training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 0:  0.01568894952251021 , 0.36425648021828105 , 0.6671214188267395 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 1:  0.012278308321964526 , 0.4638472032742155 , 0.5607094133697135 



...

Iter 2:  0.03206002728512958 , 0.4297407912687585 , 0.6343792633015006 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 3:  0.008185538881309684 , 0.5184174624829468 , 0.4979536152796726 



training:   0%|          | 0/2000 [00:00<?, ?it/s]

Iter 4:  0.017735334242837686 , 0.40791268758526605 , 0.6275579809004093 

sine:
Final Score:  0.017189631650750336 ± 0.011258703795828759



In [7]:
### Predictive score
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training:   0%|          | 0/5000 [00:00<?, ?it/s]

0  epoch:  0.03706756796237588 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

1  epoch:  0.03698619081686294 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

2  epoch:  0.03708864965207218 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

3  epoch:  0.03706569080127474 



training:   0%|          | 0/5000 [00:00<?, ?it/s]

4  epoch:  0.03723624610144554 

sine:
Final Score:  0.037088869066806265 ± 0.00011321264395846054

